[View in Colaboratory](https://colab.research.google.com/github/ZackPashkin/Evaluating-a-Data-Model/blob/master/Exploratory_Data_Analysis_Datalab,_BigQuery.ipynb)

In [0]:
!pip install --upgrade google-cloud-bigquery

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

client = bigquery.Client()


We'll use a random value in a BigQuery query to select about 1% of the dataset. We don't need to look at the whole dataset of 6 million records; a 1% sample is statistically significant enough for this analysis

In [0]:
sql = """
SELECT ARR_DELAY, DEP_DELAY
FROM `flights.tzcorr`
WHERE DEP_DELAY >= 10 AND RAND() < 0.01
"""
df = client.query(sql).to_dataframe()
df.describe()


 display a Seaborn violin chart that shows the distribution dataframe that contains the 1% sample of delay times for flights that departed 10 or more minutes late:

In [0]:
sns.set_style("whitegrid")
sns.set(font="DejaVu Sans")
ax = sns.violinplot(data=df, x='ARR_DELAY', inner='box', orient='h')

So now look at the difference between flights that depart more than 10 minutes late and those that depart less than 10 minutes late. We will add a calculated boolean variable called ‘ontime' that allows us to compare this distribution of the arrival time of flights that departed 10 or more minutes late with the distribution for flights that departed less than 10 minutes late.

In [0]:
sql = """
SELECT ARR_DELAY, DEP_DELAY
FROM `flights.tzcorr`
WHERE RAND() < 0.001
"""

df = client.query(sql).to_dataframe()
df['ontime'] = df['DEP_DELAY'] < 10
ax = sns.violinplot(data=df, x='ARR_DELAY', y='ontime',
                   inner='box', orient='h')
ax.set_xlim(-50,200);


The top plot is clipped because Seaborn's default smoothing parameter is too coarse. In order to fix this we need to pass in a more appropriate gridsize parameter. Since we're interested in the data on the left side of this plot, we can narrow down the x axis limits.

In [0]:
sql = """
SELECT ARR_DELAY, DEP_DELAY
FROM `flights.tzcorr`
WHERE RAND() < 0.001"""

df = client.query(sql).to_dataframe()
df['ontime'] = df['DEP_DELAY'] < 10
ax = sns.violinplot(data=df, x='ARR_DELAY', y='ontime',
                   inner='box', orient='h',gridsize=1000)
ax.set_xlim(-50,50);

This clearly shows that the 10 minute departure delay threshold definitely divides the dataset into two very different datasets, which is clearly shown by the box plots in the center of these two distributions. Flights that depart 10 or more minutes late are clearly asymmetrically distributed towards longer delay times. Flights that depart less than 10 minutes late are centered around the peak arrival time.

One approach to eliminating outliers from the dataset is to filter the dataset so that you only look at delay values within a particular range. For a dataset that has a few hundred thousand examples we can look at a range based on the standard deviation of the value we are interested in.

In [0]:
sql = """

SELECT
  AVG(DEP_DELAY) - 3*STDDEV(DEP_DELAY) AS filtermin,
  AVG(DEP_DELAY) + 3*STDDEV(DEP_DELAY) AS filtermax
FROM
  `flights.tzcorr`
"""
df = client.query(sql).to_dataframe()
df.describe()

In [0]:
depdelayquery = """
SELECT
  DEP_DELAY,
  arrival_delay,
  stddev_arrival_delay,
  numflights
FROM (
  SELECT
    DEP_DELAY,
    AVG(ARR_DELAY) AS arrival_delay,
    STDDEV(ARR_DELAY) AS stddev_arrival_delay,
    COUNT(ARR_DELAY) AS numflights
  FROM
    `flights.tzcorr`
  GROUP BY
    DEP_DELAY )
WHERE
  numflights > 370
ORDER BY
  DEP_DELAY
"""
depdelay = client.query(depdelayquery).to_dataframe()
depdelay.head()

plot average arrival delay and the standard deviation for each delay:

In [0]:
ax = depdelay.plot(kind='line',
                   x='DEP_DELAY',
                   y='arrival_delay',
                   yerr='stddev_arrival_delay');

This demonstrates that there does appear to be a linear relationship between departure delay and arrival delay. It also shows that the standard deviation of the arrival delay for each departure delay is very uniform and close to 10 minutes over the whole range.

plot the intersection of the 15 minute delay line with the 30% arrival delay probability line.

In [0]:
Z_30 = 0.52
depdelay['arr_delay_30'] = (
  (Z_30 * depdelay['stddev_arrival_delay'])
  + depdelay['arrival_delay']
)
plt.axhline(y=15, color='r')
ax = plt.axes()
depdelay.plot(kind='line',
              x='DEP_DELAY',
              y='arr_delay_30',
              ax=ax,
              ylim=(0, 30),
              xlim=(0, 30),
              legend=False)
ax.set_xlabel('Departure Delay (minutes)')
ax.set_ylabel('> 30% likelihood of this Arrival Delay (minutes)');

The following query breaks up the arrival delays for each departure delay into 100 bins and then selects the arrival delay for the 70th bin as the appropriate value. The 70th bin contains the value that will occur 30% of the time. We then chart the results again to find the intersection of the 15 minute delay line with the 30% probability line we have now established empirically.

In [0]:
depdelayquery2 = """
SELECT
  DEP_DELAY,
  APPROX_QUANTILES(ARR_DELAY, 101)[OFFSET(70)] AS arrival_delay,
  COUNT(ARR_DELAY) AS numflights
FROM
  `flights.tzcorr`
GROUP BY
  DEP_DELAY
HAVING
  numflights > 370
ORDER BY
  DEP_DELAY
"""
depdelay = client.query(depdelayquery2).to_dataframe()
plt.axhline(y=15, color='r')
ax = plt.axes()
depdelay.plot(kind='line',
              x='DEP_DELAY',
              y='arrival_delay',
              ax=ax,
              ylim=(0, 30),
              xlim=(0, 30),
              legend=False)
ax.set_xlabel('Departure Delay (minutes)')
ax.set_ylabel('> 30% likelihood of this Arrival Delay (minutes)');